## 1D1V Vlasov–Ampere system

In [1]:

using Pkg
Pkg.instantiate()
using ProgressMeter, FFTW, Plots, LinearAlgebra
using BenchmarkTools, Statistics

"""
    UniformMesh(start, stop, length)

1D uniform mesh data for periodic domain (end point is removed)
"""
struct UniformMesh

   start    :: Float64
   stop     :: Float64
   length   :: Int64
   step     :: Float64
   points   :: Vector{Float64}

   function UniformMesh(start, stop, length)

       points = range(start, stop=stop, length=length+1)[1:end-1]
       step = (stop - start) / length

       new( start, stop, length, step, points)

   end

end

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 %39.1 %Fetching: [==================>                      ]  44.3 %Fetching: [============================>            ]  68.7 %

UniformMesh

## Compute charge density ρ(x)

In [2]:
"""
    compute_rho( mesh, f)

Compute charge density ρ(x,t) = ∫ f(x,v,t) dv

returns ρ - ρ̄

"""
function compute_rho(meshv::UniformMesh, f)

   dv  = meshv.step
   ρ = dv .* vec(sum(real(f), dims=2))
   ρ .- mean(ρ)

end

compute_rho

## Compute electric field from ρ(x)

In [3]:
"""
    compute_e(mesh, ρ)

compute electric field from ρ
"""
function compute_e(mesh::UniformMesh, ρ)

   n = mesh.length
   k =  2π / (mesh.stop - mesh.start)
   modes = [1.0; k .* vcat(1:n÷2-1,-n÷2:-1)...]
   ρ̂ = fft(ρ)./modes
   vec(real(ifft(-1im .* ρ̂)))

end

compute_e

## Callable struct `Advection`

In [4]:
"""
    advection! = AmpereAdvection( mesh, kx)

For every created struct, two methods are available
- Advection method along v
- Advection method along x and e computation

"""
struct AmpereAdvection

    mesh :: UniformMesh
    kx   :: Vector{Float64}

    function AmpereAdvection( mesh )

        nx  = mesh.length
        dx  = mesh.step
        Lx  = mesh.stop - mesh.start
        kx  = zeros(Float64, nx)
        kx .= 2π/Lx .* [0:nx÷2-1;-nx÷2:-1]
        new( mesh, kx)

    end

end

AmpereAdvection

In [5]:
function (adv :: AmpereAdvection)( fᵗ  :: Array{ComplexF64,2},
                                   e   :: Vector{ComplexF64},
                                   dt  :: Float64 )
    fft!(fᵗ, 1)
    fᵗ .= fᵗ .* exp.(-1im * dt * adv.kx * transpose(e))
    ifft!(fᵗ, 1)

end

In [6]:
function (adv :: AmpereAdvection)( f   :: Array{ComplexF64,2},
                                   e   :: Vector{ComplexF64},
                                   v   :: Vector{Float64},
                                   dt  :: Float64,
                                   E_ext :: Vector{ComplexF64})

    
    ev = exp.(-1im*dt * adv.kx * transpose(v))
    fft!(f,1)
    f .= f .* ev
    dv = v[2]-v[1]
    ρ = dv * vec(sum(f,dims=2))
    for i in 2:length(e)
        e[i] = -1im * ρ[i] / adv.kx[i]
    end
    e[1] = 0.0
    ifft!(f, 1)
    ifft!(e)
    e .= real(e) + real(E_ext) #External Electric field superposed
end

## Initial distribution function

In [7]:
"""
    landau( ϵ, kx, x, v )

Landau damping initialisation function

[Wikipedia](http
s://en.wikipedia.org/wiki/Landau_damping)


"""
function landau( ϵ, kx, x, v )
   (1.0 .+ ϵ*cos.(kx*x))/sqrt(2π) .* transpose(exp.(-0.5*v.*v))
end


nx, nv = 512, 512
xmin, xmax = -0.0*π, 4*π
vmin, vmax = -6., 6.
tf = 100
nt = 500
meshx = UniformMesh(xmin, xmax, nx)
meshv = UniformMesh(vmin, vmax, nv);

Initialize distribution function

In [8]:
x = meshx.points
v = meshv.points
ϵ, kx = 0.001, 0.5

(0.001, 0.5)

Allocate arrays for distribution function and its transposed

In [9]:
f = zeros(Complex{Float64},(nx,nv))
fᵀ= zeros(Complex{Float64},(nv,nx))
f_0 = landau( ϵ, kx, x, v);
f .= landau( ϵ, kx, x, v);

Plot the distribution

In [10]:
heatmap(v, x ,real((f)), xlabel=:v, ylabel=:x,title="Intial Distribution f(x,v,t=0)", label= "Landau damping")
savefig("./test/initial_RF")

The external electric field can now be specified, in this case an electrostatic driving wave $E_{ext}(x,t) = E_0\exp(i(kx-\omega t))$

In [11]:
function E_RF(E_0,w,x,dt,k_t,i)
    E_0*exp.(1im*(k_t.*x.-w*i*dt)) 
end

E_RF (generic function with 1 method)

In [12]:

#Parameters for external field
E_0 = 0.01
L = x[end]-x[1]
n = 1
k_t = 2π/(L)
w = 1


dt = tf / nt

#Value of External field at t = 0
E_ext0 = E_RF(E_0,w,x,dt,k_t,0)

transpose!(fᵀ,f)
ρ  = compute_rho(meshv, f)
e  = zeros(ComplexF64, nx)
e .= compute_e(meshx, ρ) + E_ext0 #External field superposed with initial field

mod_e = Float64[]

EE = Float64[] #Electrical potential energy array
EK = Float64[] #Kinetic energy of particles array
ENTROPY = Float64[] #Gibbs entropy array
mod_ext = Float64[] 

advection_x! = AmpereAdvection( meshx )
advection_v! = AmpereAdvection( meshv );

p = heatmap(x,v,real(f))

@gif for i in 1:nt
    
    global E_ext = E_RF(E_0,w,x,dt,k_t,i)
    
    advection_v!(fᵀ, e, 0.5dt)

    transpose!(f, fᵀ)

    advection_x!( f, e, v, dt, E_ext)
        
    push!(mod_e, (sqrt(sum(e.^2)*meshx.step)))
    
    transpose!(fᵀ, f)

    advection_v!(fᵀ, e, 0.5dt)
    
    push!(mod_ext, (sqrt(sum(real(E_ext).^2)*meshx.step)))
    
    push!(EE, real(sum((sqrt.(e.^2)).^2)*meshx.step))
    
    push!(EK, sum(real((sum((f.*(v.^2)*meshv.step))*meshx.step))))
    
    push!(ENTROPY, -sum(real(sum(f.*log.(abs.(f)))*meshx.step)*meshv.step))
    
    t_1 = i*dt
    
    fv = vec(sum(real((f)).*meshx.step,dims=1))
    
    #Can make an animation of the f(x,v,t) heatmap or the f(v,t) plot by uncommenting desired
    #plot
    
    plot(v,fv, xlabel="v", ylabel ="f(v,t)", title = "Projection of f(x,v,t) on to Velocity Space at Time t = $t_1",label = "Velocity Distribution of Particles")
    
    #heatmap(v, x, (real(f)), xlabel="v",ylabel ="x", title = "f(x,v,t) at Time t = $t_1")
    
end

t = range(0, stop=tf, length=nt)

plot(t, mod_e,title = "Electric Field E_1(x,t) + E_ext(x,t) vs Time" ,label= "Electric Field",xlabel = "Time" ,legend=:bottomright,ylabel = "Magnitude of E(t)")
savefig("./E_1.png")

plot(t,mod_e, title="E(x,t) and E_ext(x,t) vs time", xlabel=:t, ylabel= "Magnitude of Electric Field", label = " Magnitude of total field E(x,t)")
plot!(t,mod_ext, label = "Magnitude of E_ext(x,t)")
savefig("./comparison.png")

plot(t,EE+EK,label = "Electrostatic Potential Energy + Kinetic Energy of Particles",title = "Total Energy vs Time", xlabel = "Time", ylabel = "EE+EK",legend=:bottomright)
savefig("./Energy.png")

plot(t,ENTROPY, title ="Entropy vs Time",label = "Gibbs Entropy of Particle Distribution", xlabel = "Time",ylabel = "Entropy",legend=:bottomright)
savefig("./Entropy.png")

┌ Info: Saved animation to 
│   fn = /home/paul/Numkin2019/notebooks/tmp.gif
└ @ Plots /home/paul/.julia/packages/Plots/qZHsp/src/animation.jl:98


0.0:0.20040080160320642:100.0